<a href="https://colab.research.google.com/github/solon-exe/reconhecimento-facial-pipeline/blob/main/Conhe%C3%A7a_o_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files

# Faça o upload de TODOS os arquivos necessários de uma só vez:
# - face_detection.pb
# - face_classifier.h5
# - imagens de teste (.jpg, .png etc.)
uploaded = files.upload()
# Depois de subir, eles ficarão disponíveis em /content/<nome_do_arquivo>

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
DETECT_MODEL_PATH = 'face_detection.pb'   # arquivo que você fez upload

detector = cv2.dnn.readNetFromTensorflow(DETECT_MODEL_PATH)

In [ ]:
def detect_faces(img, conf_threshold=0.5):
    h, w = img.shape[:2]
    blob = cv2.dnn.blobFromImage(img, 1.0, (300,300), (104,117,123), swapRB=False)
    detector.setInput(blob)
    detections = detector.forward()
    boxes = []
    for i in range(detections.shape[2]):
        score = detections[0,0,i,2]
        if score > conf_threshold:
            box = detections[0,0,i,3:7] * np.array([w,h,w,h])
            x1,y1,x2,y2 = box.astype(int)
            boxes.append([x1,y1,x2,y2])
    return boxes

In [ ]:
W, H = 160, 160   # ajuste para o seu classificador

def extract_face(img, box):
    x1, y1, x2, y2 = box
    face = img[y1:y2, x1:x2]
    face = cv2.resize(face, (W, H))
    face = face.astype('float32') / 255.0
    return np.expand_dims(face, axis=0)

In [ ]:
CLASS_MODEL_PATH = 'face_classifier.h5'   # arquivo que você fez upload
classifier = tf.keras.models.load_model(CLASS_MODEL_PATH)

# Ajuste a lista abaixo de acordo com suas classes
CLASS_NAMES = ['amy','raj','penny','sheldon','leonard','bernadette']

In [ ]:
def classify_face(face_tensor):
    preds = classifier.predict(face_tensor)[0]
    idx = np.argmax(preds)
    return CLASS_NAMES[idx], float(preds[idx])

In [ ]:
# Ajuste o nome da imagem de teste que você fez upload
img = cv2.imread('grupo.jpg')
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

boxes = detect_faces(img)
for box in boxes:
    face = extract_face(img_rgb, box)
    name, score = classify_face(face)
    x1,y1,x2,y2 = box
    cv2.rectangle(img_rgb, (x1,y1), (x2,y2), (0,255,0), 2)
    cv2.putText(img_rgb, f"{name} ({score:.2f})", (x1, y1-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)

plt.figure(figsize=(10,8))
plt.imshow(img_rgb)
plt.axis('off')
plt.show()

In [ ]:
# Salva imagem final no ambiente do Colab
plt.imsave('resultado_pipeline.jpg', img_rgb)